In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import pathlib, requests, re

In [3]:
pathBLT = pathlib.Path('../data/raw/others/BLT.xlsx')
assert pathBLT.exists()

In [4]:
dictBLT = dict()

for year in range(1997, pd.datetime.today().year+1):
    foundInYear = 0
    for month in range(1, 12+1):
        if (year, month) not in {(1998, 11), (2001, 3)}:
            
            yearmonth = str(year) + str(month).zfill(2)
            if int(yearmonth) < 201703:
                url = f'https://www.federalreserve.gov/boarddocs/SnLoanSurvey/{yearmonth}/default.htm'
            elif int(yearmonth) < 201801:
                url = f'https://www.federalreserve.gov/data/sloos/{yearmonth}/default.htm'
            else:
                url = f'https://www.federalreserve.gov/data/sloos/sloos-{yearmonth}.htm'

            page = requests.get(url)
            if page.status_code == 200:
                foundInYear +=1
                dictBLT[(year, month)] = page.content

    print(f'year {year}: {foundInYear} reports')

year 1997: 4 reports
year 1998: 4 reports
year 1999: 4 reports
year 2000: 4 reports
year 2001: 4 reports
year 2002: 4 reports
year 2003: 4 reports
year 2004: 4 reports
year 2005: 4 reports
year 2006: 4 reports
year 2007: 4 reports
year 2008: 4 reports
year 2009: 4 reports
year 2010: 4 reports
year 2011: 4 reports
year 2012: 4 reports
year 2013: 4 reports
year 2014: 4 reports
year 2015: 4 reports
year 2016: 4 reports
year 2017: 4 reports
year 2018: 4 reports
year 2019: 4 reports
year 2020: 3 reports


In [84]:
vintageDates = []
for (year, month), value in dictBLT.items():

    page = BeautifulSoup(value, 'html.parser').text.lower().replace('\r', ' ').replace('\n', ' ')
    page = re.sub('( ){1,}', ' ', page)
    match = re.search('last update: (([a-z]){1,10} ([0-9]){1,2}, ([0-9]){4})', page)

    if match is None:
        print((year, month))
        break
    else:
        print(match.group(0))
        if match.group(1) == 'august 27, 1999':
            vintageDate = 'may 21, 1999'
        elif match.group(1) == 'august 22, 2005' and month == 5:
            vintageDate = 'may 9, 2005'
        elif match.group(1) == 'august 22, 2005' and month == 8:
            vintageDate = 'august 15, 2005'
        elif match.group(1) == 'june 06, 2019':
            vintageDate = 'may 08, 2018'
        else:
            vintageDate = match.group(1)

        vintageDates.append(pd.to_datetime(vintageDate))

last update: february 10, 1997
last update: may 23, 1997
last update: august 22, 1997
last update: november 17, 1997
last update: february 9, 1998
last update: may 26, 1998
last update: august 21, 1998
last update: october 1, 1998
last update: february 8, 1999
last update: august 27, 1999
last update: august 30, 1999
last update: november 22, 1999
last update: february 8, 2000
last update: may 19, 2000
last update: august 25, 2000
last update: november 17, 2000
last update: february 5, 2001
last update: may 17, 2001
last update: august 24, 2001
last update: november 13, 2001
last update: february 4, 2002
last update: may 10, 2002
last update: august 19, 2002
last update: november 12, 2002
last update: january 31, 2003
last update: may 9, 2003
last update: august 15, 2003
last update: november 3, 2003
last update: february 3, 2004
last update: may 7, 2004
last update: august 16, 2004
last update: november 15, 2004
last update: february 7, 2005
last update: august 22, 2005
last update: a

In [86]:
seriesBLT

,BLT
1994-01-01,-8.870000
1994-04-01,-8.832500
1994-07-01,-5.637500
1994-10-01,-10.862500
1995-01-01,-3.375000
...,...
2019-04-01,0.748810
2019-07-01,-1.330952
2019-10-01,6.700000
2020-01-01,-0.030952


In [90]:
seriesBLT = pd.read_excel(pathBLT, sheet_name='series', encoding='utf-8', index_col=0)

In [91]:
dictVintageDateReplace = {
    pd.to_datetime('1999-08-27'): pd.to_datetime('1999-05-21')
}

In [92]:
dfBLT = pd.DataFrame(index = seriesBLT.index)

for vintageDate in vintageDates:
    column = 'BLT_'+vintageDate.strftime('%Y%m%d')
    dfBLT[column] = seriesBLT['BLT'].values
    for row in seriesBLT.index:
        if row.to_period('Q') > vintageDate.to_period('Q'):
            dfBLT.loc[row, column] = float('nan')

In [100]:
dfBLT.index = dfBLT.index.map(lambda x: x.strftime('%m/%d/%Y').lstrip('0').replace('/0', '/'))
with pd.ExcelWriter(pathBLT2) as writer:
    dfBLT.to_excel(writer, sheet_name='dataframe')